## Data Cleaning:

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import duckdb
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import plotly.express as px

### Happiness table

First we loaded in the Happiness CSV file

In [25]:
#load the world happiness score of 2022
happiness_df = pd.read_csv('2022.csv', thousands=',')
print(happiness_df.shape)

(147, 12)


The original dataset displays the world happiness country ranking of 2022 and scores from various perspectives. We removed the rows with empty data and are only keeping the happiness rank, happiness score, and country name for further analysis.

In [26]:

#drop the rows where any of these columns is empty.
happiness_df = happiness_df.dropna(subset=['Country', 'Happiness score'])

#select the 'Happiness rank' and 'Country' columns from the dataset
happiness_df = happiness_df[['Country', 'Happiness score']]

print(happiness_df.shape)
happiness_df

(146, 2)


,Country,Happiness score
0,Finland,7821.0
1,Denmark,7636.0
2,Iceland,7557.0
3,Switzerland,7512.0
4,Netherlands,7415.0
...,...,...
141,Botswana*,3471.0
142,Rwanda*,3268.0
143,Zimbabwe,2995.0
144,Lebanon,2955.0


### Economy and Crime

Note: we are printing dataframe below because there are so many columns that it goes off the page when presented as a table. Printing allows for all the columns of data to be visible. For this project, we will print the dataframes that have columns going off the page. For the other dataframes, we will present them as tables since that provides for easier viewing.

In [27]:
#load the economic data of countries for 2023
econ_df = pd.read_csv('world-data-2023.csv', thousands=',')
print(econ_df.shape)
print(econ_df)

(195, 35)
         Country  Density\n(P/Km2) Abbreviation Agricultural Land( %)  \
0    Afghanistan                60           AF                58.10%   
1        Albania               105           AL                43.10%   
2        Algeria                18           DZ                17.40%   
3        Andorra               164           AD                40.00%   
4         Angola                26           AO                47.50%   
..           ...               ...          ...                   ...   
190    Venezuela                32           VE                24.50%   
191      Vietnam               314           VN                39.30%   
192        Yemen                56           YE                44.60%   
193       Zambia                25           ZM                32.10%   
194     Zimbabwe                38           ZW                41.90%   

     Land Area(Km2)  Armed Forces size  Birth Rate  Calling Code  \
0          652230.0           323000.0       

The original dataset includes basic information about 195 countries with 35 attributes for each country, such as population density, abbreviation, and much more. We are only using information regarding the economy from this dataset, such as GDP, to analyze how economic factors within countries affect citizens’ happiness. We cleaned the rows with missing data and renamed some of the columns to clarify the unit and explain the economic terms. We created a column called ‘GDP Per Capita’ based on the provided GDP and Population information and divided them to gain this essential metric of economics. In addition, we removed special signs such as dollar signs and percentage signs and converted them to numbers for later analysis. We ended up with data for 128 countries.

In [28]:
#calculate the GDP Per Capita 
econ_df['GDP'] = econ_df['GDP'].str.replace(',', '') #remove commas
econ_df['GDP'] = econ_df['GDP'].str.replace('$', '') #remove dollar sign 
econ_df['GDP'] = pd.to_numeric(econ_df['GDP'], errors='coerce') #convert to numeric
econ_df['GDP Per Capita'] = econ_df['GDP'] / econ_df['Population']

econ_df['Minimum wage'] = econ_df['Minimum wage'].str.replace('$', '') #remove dollar sign 
econ_df['Minimum wage'] = pd.to_numeric(econ_df['Minimum wage'], errors='coerce') #convert to numeric

econ_df['Tax revenue (%)'] = econ_df['Tax revenue (%)'].str.replace('%', '') #remove percentage sign
econ_df['Tax revenue (%)'] = pd.to_numeric(econ_df['Tax revenue (%)'], errors='coerce') #convert to numeric
econ_df['Unemployment rate'] = econ_df['Unemployment rate'].str.replace('%', '') #remove percentage sign

#rename the columns for better clarification 
econ_df.rename(columns={'CPI': 'Consumer Price Index'}, inplace=True)
econ_df.rename(columns={'Unemployment rate': 'Unemployment rate(%)'}, inplace=True)
econ_df.rename(columns={'GDP': 'GDP($)'}, inplace=True)
econ_df.rename(columns={'Minimum wage': 'Minimum wage($)'}, inplace=True)

#select the columns related to the economy factor
econ_df = econ_df[['Country', 'GDP($)', 'Population', 'GDP Per Capita',
                   'Minimum wage($)', 'Unemployment rate(%)',
                   'Consumer Price Index', 'Tax revenue (%)']]

#drop the rows where any of these columns is empty.
econ_df = econ_df.dropna(subset=['Country', 'GDP($)', 'Population', 
                                 'GDP Per Capita','Minimum wage($)', 
                                 'Unemployment rate(%)','Consumer Price Index', 
                                 'Tax revenue (%)'])

print(econ_df.shape)
econ_df

(128, 8)


,Country,GDP($),Population,GDP Per Capita,Minimum wage($),Unemployment rate(%),Consumer Price Index,Tax revenue (%)
0,Afghanistan,1.910135e+10,38041754.0,502.115487,0.43,11.12,149.90,9.3
1,Albania,1.527808e+10,2854191.0,5352.857411,1.12,12.33,119.05,18.6
2,Algeria,1.699882e+11,43053054.0,3948.343279,0.95,11.70,151.36,37.2
4,Angola,9.463542e+10,31825295.0,2973.591160,0.71,6.89,261.73,9.2
6,Argentina,4.496634e+11,44938712.0,10006.148974,3.35,9.79,232.75,10.1
...,...,...,...,...,...,...,...,...
186,United States,2.142770e+13,328239523.0,65280.682241,7.25,14.70,117.24,9.6
187,Uruguay,5.604591e+10,3461734.0,16190.126957,1.66,8.73,202.92,20.1
189,Vanuatu,9.170589e+08,299882.0,3058.065676,1.56,4.39,117.13,17.8
191,Vietnam,2.619212e+11,96462106.0,2715.276036,0.73,2.01,163.52,19.1


Next we loaded the crime data for countries during the year 2023

In [29]:
#load the crime data of countries for 2023
crime_df = pd.read_csv('crime-rate-by-country-2023.csv', thousands=',')
print(crime_df.shape)
crime_df

(133, 4)


,rank,country,crimeIndex,pop2023
0,1,Venezuela,83.76,28838499.0
1,2,Papua New Guinea,80.79,10329931.0
2,3,South Africa,76.86,60414495.0
3,4,Afghanistan,76.31,42239854.0
4,5,Honduras,74.54,10593798.0
...,...,...,...,...
128,129,Switzerland,21.62,8796669.0
129,130,Oman,20.34,4644384.0
130,131,Taiwan,15.46,23923276.0
131,132,United Arab Emirates,15.23,9516871.0


For the Crime dataset, we are renaming the ‘rank’ column to ‘Crime rank’ to avoid conflict with ‘Happiness rank’ after merging the dataframes. We are also renaming ‘country’ to ‘Country’ so that we can merge these three dataframes based on the ‘Country’ column. Additionally, we are renaming ‘crimeIndex’ to ‘Crime Index’ to keep the first letter capitalized for consistency. We are deleting the ‘pop2023’ column as it will be repetitive information. We are again dropping rows with empty data, but in this case, none of them are dropped.

In [30]:
#rename columns
crime_df.rename(columns={'country': 'Country'}, inplace=True) 
crime_df.rename(columns={'crimeIndex': 'Crime Index'}, inplace=True)
crime_df.rename(columns={'rank': 'Crime rank'}, inplace=True)

#select columns
crime_df=crime_df[['Country','Crime rank','Crime Index']]

#drop the rows where any of these columns is empty.
crime_df=crime_df.dropna(subset=['Country', 'Crime Index','Crime rank'])

print(crime_df.shape)
crime_df

(133, 3)


,Country,Crime rank,Crime Index
0,Venezuela,1,83.76
1,Papua New Guinea,2,80.79
2,South Africa,3,76.86
3,Afghanistan,4,76.31
4,Honduras,5,74.54
...,...,...,...
128,Switzerland,129,21.62
129,Oman,130,20.34
130,Taiwan,131,15.46
131,United Arab Emirates,132,15.23


After merging the three DataFrames (econ_df, crime_df, and happiness_df) using the 'Country' column with inner joins, we now have data for analysis from 81 countries.

In [31]:
#merge three dataframes into a single data set 
merged_df = pd.merge(econ_df, crime_df, on='Country', how='inner')
merged_df = pd.merge(merged_df, happiness_df, on='Country', how='inner')
print(merged_df.shape)
merged_df

(81, 11)


,Country,GDP($),Population,GDP Per Capita,Minimum wage($),Unemployment rate(%),Consumer Price Index,Tax revenue (%),Crime rank,Crime Index,Happiness score
0,Afghanistan,1.910135e+10,38041754.0,502.115487,0.43,11.12,149.90,9.3,4,76.31,2404.0
1,Albania,1.527808e+10,2854191.0,5352.857411,1.12,12.33,119.05,18.6,76,42.53,5199.0
2,Algeria,1.699882e+11,43053054.0,3948.343279,0.95,11.70,151.36,37.2,42,52.03,5122.0
3,Argentina,4.496634e+11,44938712.0,10006.148974,3.35,9.79,232.75,10.1,17,63.82,5967.0
4,Armenia,1.367280e+10,2957731.0,4622.733493,0.66,16.99,129.18,20.9,125,22.79,5399.0
...,...,...,...,...,...,...,...,...,...,...,...
76,United Kingdom,2.827113e+12,66834405.0,42300.267126,10.13,3.85,119.62,25.5,63,46.07,6943.0
77,United States,2.142770e+13,328239523.0,65280.682241,7.25,14.70,117.24,9.6,55,47.81,6977.0
78,Uruguay,5.604591e+10,3461734.0,16190.126957,1.66,8.73,202.92,20.1,44,51.73,6474.0
79,Vietnam,2.619212e+11,96462106.0,2715.276036,0.73,2.01,163.52,19.1,62,46.19,5485.0


### Demography
Here we will import a dataframe that contains information about demography factors across different countries. This will be compared to countries' happiness scores to understand if there are any trends or patterns.

In [32]:
world_data_df = pd.read_csv("world-data-2023.csv", thousands = ",")
print(world_data_df)

         Country  Density\n(P/Km2) Abbreviation Agricultural Land( %)  \
0    Afghanistan                60           AF                58.10%   
1        Albania               105           AL                43.10%   
2        Algeria                18           DZ                17.40%   
3        Andorra               164           AD                40.00%   
4         Angola                26           AO                47.50%   
..           ...               ...          ...                   ...   
190    Venezuela                32           VE                24.50%   
191      Vietnam               314           VN                39.30%   
192        Yemen                56           YE                44.60%   
193       Zambia                25           ZM                32.10%   
194     Zimbabwe                38           ZW                41.90%   

     Land Area(Km2)  Armed Forces size  Birth Rate  Calling Code  \
0          652230.0           323000.0       32.49     

Now we rename our columns of interest to have proper syntax so they are easy to access. The factors we are interested in looking at are birth rate, infant mortality, life expectancy, and maternal mortality ratio.

In [33]:
world_data_df = world_data_df.rename(columns={'Birth Rate': 'birth_rate', 
                'Infant mortality': 'infant_mortality', 
                'Life expectancy': 'life_expectancy', 
                'Maternal mortality ratio': 'maternal_mortality_ratio'})
print(world_data_df)

         Country  Density\n(P/Km2) Abbreviation Agricultural Land( %)  \
0    Afghanistan                60           AF                58.10%   
1        Albania               105           AL                43.10%   
2        Algeria                18           DZ                17.40%   
3        Andorra               164           AD                40.00%   
4         Angola                26           AO                47.50%   
..           ...               ...          ...                   ...   
190    Venezuela                32           VE                24.50%   
191      Vietnam               314           VN                39.30%   
192        Yemen                56           YE                44.60%   
193       Zambia                25           ZM                32.10%   
194     Zimbabwe                38           ZW                41.90%   

     Land Area(Km2)  Armed Forces size  birth_rate  Calling Code  \
0          652230.0           323000.0       32.49     

Here we modify our dataframe to include only the columns that are relevant for our analysis. This results in a cleaner, less cluttered dataframe. Columns with NaN values are dropped, so that there are no missing data values present.

In [34]:
world_data_df = duckdb.sql("SELECT Country, birth_rate, infant_mortality, \
life_expectancy, maternal_mortality_ratio FROM world_data_df").df()
world_data_df = world_data_df.dropna()
world_data_df

,Country,birth_rate,infant_mortality,life_expectancy,maternal_mortality_ratio
0,Afghanistan,32.49,47.9,64.5,638.0
1,Albania,11.78,7.8,78.5,15.0
2,Algeria,24.28,20.1,76.7,112.0
4,Angola,40.73,51.6,60.8,241.0
5,Antigua and Barbuda,15.33,5.0,76.9,42.0
...,...,...,...,...,...
190,Venezuela,17.88,21.4,72.1,125.0
191,Vietnam,16.75,16.5,75.3,43.0
192,Yemen,30.45,42.9,66.1,164.0
193,Zambia,36.19,40.4,63.5,213.0


Next, we will refer back to the Happiness Score we used previously

In [35]:
happiness_df

,Country,Happiness score
0,Finland,7821.0
1,Denmark,7636.0
2,Iceland,7557.0
3,Switzerland,7512.0
4,Netherlands,7415.0
...,...,...
141,Botswana*,3471.0
142,Rwanda*,3268.0
143,Zimbabwe,2995.0
144,Lebanon,2955.0


Here, we join the world data dataframe, which contains relevant demography statistics by country, to the world happiness dataframe, which includes happiness scores by country. This creates a single dataframe with all the relevant information for analysis between demography statistics and happiness.

In [36]:
demography_df = duckdb.sql("SELECT world_data_df.Country, birth_rate, \
infant_mortality, life_expectancy, maternal_mortality_ratio, \
\"Happiness score\" FROM world_data_df INNER JOIN happiness_df \
ON world_data_df.Country = happiness_df.Country").df()
demography_df

,Country,birth_rate,infant_mortality,life_expectancy,maternal_mortality_ratio,Happiness score
0,Afghanistan,32.49,47.9,64.5,638.0,2404.0
1,Albania,11.78,7.8,78.5,15.0,5199.0
2,Algeria,24.28,20.1,76.7,112.0,5122.0
3,Armenia,13.99,11.0,74.9,26.0,5399.0
4,Australia,12.60,3.1,82.7,6.0,7162.0
...,...,...,...,...,...,...
113,Kazakhstan,21.77,8.8,73.2,10.0,6234.0
114,Lithuania,10.00,3.3,75.7,8.0,6446.0
115,Nigeria,37.91,75.7,54.3,917.0,4552.0
116,Romania,9.60,6.1,75.4,19.0,6477.0


Additionally, we are adding some regional data, specifically continent and subregion, to help categorize groups of countries.

In [37]:
continent_df = pd.read_csv("countryContinent.csv", encoding='ISO-8859-1')
continent_df

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0
...,...,...,...,...,...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,9.0,61.0
245,Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,2.0,15.0
246,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,142.0,145.0
247,Zambia,ZM,ZMB,894,ISO 3166-2:ZM,Africa,Eastern Africa,2.0,14.0


The dataframe is cleaned to only include relevant variables.

In [38]:
continent_df = duckdb.sql("SELECT country, continent, sub_region FROM \
continent_df").df()
continent_df

,country,continent,sub_region
0,Afghanistan,Asia,Southern Asia
1,Åland Islands,Europe,Northern Europe
2,Albania,Europe,Southern Europe
3,Algeria,Africa,Northern Africa
4,American Samoa,Oceania,Polynesia
...,...,...,...
244,Wallis and Futuna,Oceania,Polynesia
245,Western Sahara,Africa,Northern Africa
246,Yemen,Asia,Western Asia
247,Zambia,Africa,Eastern Africa


Then it is joined to the demography data.

In [39]:
demography_df = duckdb.sql("SELECT continent_df.country, continent, \
sub_region, birth_rate, infant_mortality, life_expectancy, \
maternal_mortality_ratio, \"Happiness score\", FROM continent_df INNER JOIN \
demography_df ON continent_df.country = demography_df.Country").df()
demography_df

,country,continent,sub_region,birth_rate,infant_mortality,life_expectancy,maternal_mortality_ratio,Happiness score
0,Afghanistan,Asia,Southern Asia,32.49,47.9,64.5,638.0,2404.0
1,Albania,Europe,Southern Europe,11.78,7.8,78.5,15.0,5199.0
2,Algeria,Africa,Northern Africa,24.28,20.1,76.7,112.0,5122.0
3,Armenia,Asia,Western Asia,13.99,11.0,74.9,26.0,5399.0
4,Australia,Oceania,Australia and New Zealand,12.60,3.1,82.7,6.0,7162.0
...,...,...,...,...,...,...,...,...
101,Austria,Europe,Western Europe,9.70,2.9,81.6,5.0,7163.0
102,Ecuador,Americas,South America,19.72,12.2,76.8,59.0,5533.0
103,Lithuania,Europe,Northern Europe,10.00,3.3,75.7,8.0,6446.0
104,Romania,Europe,Eastern Europe,9.60,6.1,75.4,19.0,6477.0


The data can then be separated into dataframes by continent for further analysis.

In [57]:
hypothesis2_df = duckdb.sql("SELECT merged_df.Country, Population, \"Crime Index\" AS \"Crime index\", \"Minimum wage($)\" AS \"Minimum wage ($)\", \
birth_rate AS \"Birth rate\", demography_df.\"Happiness score\" FROM merged_df INNER JOIN demography_df \
ON merged_df.Country = demography_df.Country").df()
hypothesis2_df

,Country,Population,Crime index,Minimum wage ($),Birth rate,Happiness score
0,Afghanistan,38041754.0,76.31,0.43,32.49,2404.0
1,Albania,2854191.0,42.53,1.12,11.78,5199.0
2,Algeria,43053054.0,52.03,0.95,24.28,5122.0
3,Armenia,2957731.0,22.79,0.66,13.99,5399.0
4,Australia,25766605.0,43.03,13.59,12.60,7162.0
...,...,...,...,...,...,...
67,Kazakhstan,18513930.0,53.77,0.41,21.77,6234.0
68,Nigeria,200963599.0,64.06,0.54,37.91,4552.0
69,Argentina,44938712.0,63.82,3.35,17.02,5967.0
70,Lithuania,2786844.0,33.42,2.41,10.00,6446.0


### Health

First we load in the CSV files analyze suicide rates and mental health disorders in various countries

In [51]:
mental_health_df = pd.read_csv('Mental health Depression disorder Data.csv', dtype='unicode')
# https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options

#### Mental Disorders
We begin by removing unnecessary columns from the data frame

In [52]:
mental_health_df.drop('Code', axis=1, inplace=True)

We rename the columns to make them more intuitive. We renamed the "Entity" column to the "Country". Then we removed the missing values from the dataframe to ensure we only focus on the relevant data.

In [53]:
#Rename the Entity column to countries
mental_health_df = mental_health_df.rename(columns={'Entity': 'Country'})

# #Remove NaN
mental_health_df = mental_health_df.dropna()

The "Year" column wasn't an int, so we changed it to make it easier to remove and select years later on. We deleted years before 2013 since we wanted to collect more recent data. 

In [54]:
# Make years into int, since it was an object before
mental_health_df['Year'] = mental_health_df['Year'].astype(int)

# remove years before 2013 
mental_health_df = mental_health_df[mental_health_df['Year'] >= 2013]

The disorder columns contained errors in its data that needed to be removed. For example, one of its data contained 2 periods instead 1 for percentages. To solve this we converted all the disorders to numeric values and set all non-passing vales as NaN. We then dropped those values.

In [55]:
# Remove the data in the percentages that aren't numeric. Ex) 
columns_to_convert = [
    'Schizophrenia (%)', 'Bipolar disorder (%)', 'Eating disorders (%)',
    'Anxiety disorders (%)', 'Drug use disorders (%)',
    'Depression (%)', 'Alcohol use disorders (%)']

# change to numeric, ignore errors
# https://stackoverflow.com/questions/44522741/pandas-mean-typeerror-could-not-convert-to-numeric
mental_health_df[columns_to_convert] = mental_health_df[columns_to_convert].apply(pd.to_numeric, 
                                                                                  errors='coerce')
mental_health_df.dropna(subset=columns_to_convert, inplace=True)

We then grouped the data by country and took the mean of the disorder percentages. We dropped the columns that were not relevant to the final dataframe.

In [46]:
# Group the disorder percentage mean for 2013-2017
averages_by_country = mental_health_df.groupby('Country').mean()
averages_by_country.drop(axis=0, columns=['Year','index'], inplace=True)
averages_by_country.reset_index(inplace=True)
averages_by_country

,Country,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
0,Afghanistan,0.165882,0.707279,0.104365,4.875703,2.533182,4.135129,0.662045
1,Albania,0.200605,0.704141,0.170857,3.387972,0.512789,2.205231,1.847211
2,Algeria,0.197709,0.817824,0.211327,5.058440,1.708282,3.639566,0.665544
3,American Samoa,0.248964,0.468421,0.179106,3.316180,0.769744,2.937838,1.141303
4,Andean Latin America,0.200701,0.881123,0.350026,4.266526,0.666658,2.606462,1.548944
...,...,...,...,...,...,...,...,...
226,Western Sub-Saharan Africa,0.174619,0.636718,0.139599,2.904105,0.483082,3.757981,0.967807
227,World,0.254415,0.596277,0.207662,3.774304,0.930513,3.439976,1.412415
228,Yemen,0.172382,0.735498,0.136447,4.852630,1.400195,4.017016,0.635910
229,Zambia,0.165007,0.605270,0.138857,3.585710,0.507067,3.581924,1.632803


#### Suicide Table

First we renamed the countries column to Country to better align with the other CSV file names. Then we dropped the years before the year 2013 and reset the index for more accurate data collection. 

In [47]:
#Rename the Entity column to countries
suicide_df = pd.read_csv('master.csv')
suicide_df = suicide_df.rename(columns={'country': 'Country'})

# drop years before 2013
suicide_df = suicide_df[suicide_df['year'] >= 2013]


suicide_df.reset_index(drop=True, inplace=True)


First we added together the suicide numbers if the year and the country was the same. We did this to account for the fact that the data took data based on different age groups. Then, we collected the suicide rates for each country and retook the sum of the suicide number.

In [48]:
suicide_df.rename(columns={'suicides/100k pop': 'suicide_rate'}, inplace=True)
suicide_df = suicide_df.groupby(['Country', 'year'])['suicide_rate'].mean().reset_index()
# Total number of suicides on the most recent years
suicide_2013_to_2017_df = suicide_df.groupby('Country')['suicide_rate'].mean().reset_index()
suicide_2013_to_2017_df

,Country,suicide_rate
0,Antigua and Barbuda,0.433889
1,Argentina,9.080556
2,Armenia,3.509208
3,Australia,12.278611
4,Austria,17.425750
...,...,...
79,Ukraine,20.634167
80,United Kingdom,7.099722
81,United States,14.261389
82,Uruguay,21.326389


Next we will refer back to the Happiness score we had before

In [49]:
happiness_df

,Country,Happiness score
0,Finland,7821.0
1,Denmark,7636.0
2,Iceland,7557.0
3,Switzerland,7512.0
4,Netherlands,7415.0
...,...,...
141,Botswana*,3471.0
142,Rwanda*,3268.0
143,Zimbabwe,2995.0
144,Lebanon,2955.0


#### Merge all 3 tables by country

We merged together the suicide, mental disorder, and happiness score dateframe.

In [56]:
merged_1_2_df= pd.merge(averages_by_country, suicide_2013_to_2017_df, on='Country', how='inner')
health_df = pd.merge(merged_1_2_df, happiness_df, on='Country', how='inner')
health_df.rename(columns={'suicide_rate': 'Suicide rate'}, inplace=True)
health_df

,Country,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%),Suicide rate,Happiness score
0,Argentina,0.197961,0.772578,0.397766,6.278569,0.989317,3.657941,1.823537,9.080556,5967.0
1,Armenia,0.196526,0.716817,0.164855,2.590217,0.495973,2.740602,1.979112,3.509208,5399.0
2,Australia,0.364150,1.144993,0.942223,6.595998,2.181237,4.694260,1.509925,12.278611,7162.0
3,Austria,0.256640,0.939808,0.674277,5.345074,0.909465,3.265730,1.879006,17.425750,7163.0
4,Bahrain,0.206873,0.836296,0.269325,4.657096,1.710888,3.896715,0.731456,0.375417,6647.0
...,...,...,...,...,...,...,...,...,...,...
57,Ukraine,0.190860,0.687293,0.171669,2.941524,0.696986,4.091572,4.816161,20.634167,5084.0
58,United Kingdom,0.258671,1.086026,0.542953,4.656162,1.696709,4.123517,1.911172,7.099722,6943.0
59,United States,0.335195,0.651874,0.521523,6.651867,3.272024,4.799864,2.020849,14.261389,6977.0
60,Uruguay,0.198341,0.780724,0.392434,6.309361,0.933896,3.585823,1.181403,21.326389,6474.0


**We made them into CSV files**

In [58]:
health_df.to_csv("health1.csv", index=False)
hypothesis2_df.to_csv("hypothesis2.csv", index=False)

__Added An image that didn't show up in the pdf to the appendix__

Geography Map: 

![geography map](geographymap.png)